# Convolutional Neural Network

### Imports

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.11.0'

## Part 1 - Data Preprocessing

### Processing training dataset

**Image Augmentation**

>* Data augmentation is the addtion of data by applying `transformations` on existing training data. Techniques include `resizing, flipping, rotating, cropping, padding,` on existing data set to increase image/data diversity as well as to feed more data for training purpose. It helps to address issues like overfitting and data scarcity, and it makes the model robust with better performance

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Processing Test Set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test__set = test_datagen.flow_from_directory("dataset/test_set",
                                             target_size = (64, 64),
                                             batch_size = 32,
                                             class_mode = "binary" )

Found 2000 images belonging to 2 classes.


## Part 2 Building the CNN

### Initializing the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 Convolution layer

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, # Feature metrics 
                               activation='relu', # Rectifier 
                               input_shape=[64, 64, 3])) # Image shape   3 bcz colored images for black n white 1

### Step 2 - Pooling 

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides=2))

### Adding second convolutionl layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides=2))

### Step - 3 Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

### Step 5 - output layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1, 
                              activation="sigmoid") )  # for single class. For multi-class `softmax`

## Part 3 - Training the CNN

### Compiling CNN


In [12]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training CNN on Train set

In [13]:
%%time
cnn.fit(x = training_set, validation_data= test__set, epochs= 25)

Epoch 1/25
250/250 [==============================] - 40s 157ms/step - loss: 0.6745 - accuracy: 0.5789 - val_loss: 0.7341 - val_accuracy: 0.5655
Epoch 2/25
250/250 [==============================] - 41s 164ms/step - loss: 0.6237 - accuracy: 0.6633 - val_loss: 0.5911 - val_accuracy: 0.6965
Epoch 3/25
250/250 [==============================] - 50s 199ms/step - loss: 0.5665 - accuracy: 0.7063 - val_loss: 0.5549 - val_accuracy: 0.7295
Epoch 4/25
250/250 [==============================] - 48s 194ms/step - loss: 0.5394 - accuracy: 0.7304 - val_loss: 0.5472 - val_accuracy: 0.7305
Epoch 5/25
250/250 [==============================] - 40s 161ms/step - loss: 0.4973 - accuracy: 0.7617 - val_loss: 0.5986 - val_accuracy: 0.6860
Epoch 6/25
250/250 [==============================] - 38s 152ms/step - loss: 0.4787 - accuracy: 0.7726 - val_loss: 0.5806 - val_accuracy: 0.7215
Epoch 7/25
250/250 [==============================] - 38s 151ms/step - loss: 0.4626 - accuracy: 0.7822 - val_loss: 0.5029 - val_ac

In [14]:
%%time
5+6


CPU times: total: 0 ns
Wall time: 0 ns


11

## Part 4 - Making Single Prediction


In [16]:
import numpy as np
from keras import utils

test_image = utils.load_img('dataset/single_prediction/cat_or_dog_1.jpg', 
                            target_size = (64, 64)) # As trained need to transform test image
test_image = utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0)

1/1 [==============================] - 0s 95ms/step


In [17]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [18]:
result

array([[1.]], dtype=float32)

In [19]:
result[0][0]

1.0

In [20]:
if result[0][0] > 0.5:
    prediction = "Dog"
    
else:
    prediction = "Cat"

print(prediction)

Dog


## Model Saving

In [ ]:
cnn.summary()

In [21]:
cnn.save("Saved Model/CNN_1")

INFO:tensorflow:Assets written to: Saved Model/CNN_1\assets


INFO:tensorflow:Assets written to: Saved Model/CNN_1\assets


In [22]:
my_model = tf.keras.models.load_model("Saved Model/CNN_1")

In [23]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8